In [1]:
#install.packages("raster")
install.packages("terra")

Warning message in install.packages("terra"):
“installation of package ‘terra’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
#library("raster")
library("terra")

terra 1.7.78



In [3]:
#country polygons
#world <- readOGR("/projects/shared-buckets/leitoldv/NaturalEarth/ne_10m_admin_0_countries.shp")
world <- terra::vect("/projects/shared-buckets/leitoldv/NaturalEarth/ne_10m_admin_0_countries.shp")

#1degree tiles
#G1deg_tiles <- readOGR("/projects/shared-buckets/leitoldv/AOIs/GRID1deg_poly_52N52S.shp")
G1deg_tiles <- terra::vect("/projects/shared-buckets/leitoldv/AOIs/GRID1deg_poly_52N52S.shp")

In [4]:
unique(world$CONTINENT)

#sort(world[world$CONTINENT=='Africa',]$ADM0_A3)
#sort(world[world$CONTINENT=='Asia',]$ADM0_A3)
#sort(world[world$CONTINENT=='Europe',]$ADM0_A3)
#sort(world[world$CONTINENT=='North America',]$ADM0_A3)
sort(world[world$CONTINENT=='South America',]$ADM0_A3)
#sort(world[world$CONTINENT=='Oceania',]$ADM0_A3)


[1] "Asia"                    "South America"          
[3] "Africa"                  "Europe"                 
[5] "North America"           "Oceania"                
[7] "Antarctica"              "Seven seas (open ocean)"

[1] "ARG" "BOL" "BRA" "CHL" "COL" "ECU" "FLK" "GUY" "PER" "PRY" "SUR" "URY"
[13] "VEN"

In [5]:
#select AOI (i.e. a country)
iso3 <- c("ARG")
country <- world[world$ADM0_A3 %in% iso3,]
aoi <- country

In [13]:
### OR ###

In [7]:
#read in custom AOI from a shapefile
f_path <- "/projects/shared-buckets/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/"
#iso3 <- "BA1"
#iso3 <- "BA2"
#iso3 <- "BC1"
#iso3 <- "BC2"
iso3 <- "BM1"
#iso3 <- "BP1"
#iso3 <- "BP2"
aoi <- terra::vect(paste(f_path, iso3, ".shp", sep=""))

In [8]:
if (!identical(crs(aoi), crs(G1deg_tiles))){aoi <- project(aoi, crs(G1deg_tiles))}
intersecting <- intersect(G1deg_tiles, aoi)
  if (nrow(intersecting) == 0) {stop('no intersecting G1deg_tiles found')
  } else {
    tileindex <- intersecting$id
  }
tileindex

[1] 22432 22433 22434 22435 22438 22439 22440 22612 22613 22614 22615 22616
 [13] 22618 22619 22620 22792 22793 22794 22795 22796 22797 22798 22799 22800
 [25] 22972 22973 22974 22975 22976 22977 22978 22979 22980 23149 23150 23151
 [37] 23152 23153 23154 23155 23156 23157 23158 23159 23160 23329 23330 23331
 [49] 23332 23333 23334 23335 23336 23337 23338 23339 23340 23508 23509 23510
 [61] 23511 23512 23513 23514 23515 23516 23517 23518 23519 23520 23688 23689
 [73] 23690 23691 23692 23693 23694 23695 23696 23697 23698 23699 23870 23871
 [85] 23872 23873 23874 23875 23876 24051 24052 24053 24054 24055 24230 24231
 [97] 24232 24233 24234 24410 24411 24412 24413 24414 24588 24589 24590 24591
[109] 24592 24593 24594 24766 24767 24768 24769 24770 24771 24772 24773 24774
[121] 24945 24946 24947 24948 24949 24950 24951 24952 24953 24954 25124 25125
[133] 25126 25127 25128 25129 25130 25131 25132 25133 25302 25303 25304 25305
[145] 25306 25307 25308 25309 25310 25482 25483 25484 25485 25486 25487 25488
[157] 25661 25662 25663 25839 25840 25841 26016 26017 26018 26019 26020 26197
[169] 26198 26199 26374 26554 27090 27111 27291

In [9]:
tilenames <- rep("NA",length(tileindex))
s3path <- rep("NA",length(tileindex))
l4path <- rep("NA",length(tileindex))

tileindex_df <- data.frame(tileindex, tilenames, s3path, l4path)
for(i in 1:nrow(tileindex_df)){
    tileindex_i <- tileindex_df[i,1]
    tileindex_df[i,2] <- paste("/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
    tileindex_df[i,3] <- paste("https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
    tileindex_df[i,4] <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_",tileindex_i,"_L4A.gpkg",sep="")
    }

write.csv(tileindex_df, paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))

In [10]:
tileindex_df <- read.csv(paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))
head(tileindex_df)

,X,tileindex,tilenames,s3path,l4path
,<int>,<int>,<chr>,<chr>,<chr>
1,1,22432,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22432.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22432.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22432_L4A.gpkg
2,2,22433,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22433.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22433.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22433_L4A.gpkg
3,3,22434,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22434.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22434.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22434_L4A.gpkg
4,4,22435,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22435.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22435.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22435_L4A.gpkg
5,5,22438,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22438.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22438.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22438_L4A.gpkg
6,6,22439,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22439.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22439.geojson,/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_22439_L4A.gpkg


In [12]:
## check if all L4A.gpkg files exist for a given country -- list missing tiles for subsetting
sort(world[world$CONTINENT=='South America',]$ADM0_A3)

[1] "ARG" "BOL" "BRA" "CHL" "COL" "ECU" "FLK" "GUY" "PER" "PRY" "SUR" "URY"
[13] "VEN"

In [32]:
iso3 <- "VEN"
tindex_path <- "/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/"
tileindex_iso3 <- read.csv(paste(tindex_path,"tileindex_",iso3,".csv", sep=""))
#head(tileindex_iso3)

L4exists <- rep(0,nrow(tileindex_iso3))
for(i in 1:nrow(tileindex_iso3)){
    L4exists[i] <- as.numeric(file.exists(tileindex_iso3[i,"l4path"]))
#    gpkg_table(tileindex_iso3[i,"l4path"])
}
L4exists

missing_tiles <- tileindex_iso3[which(L4exists==0),"tileindex"]
missing_tiles

for(j in 1:length(missing_tiles)){

    print(paste("#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",
                missing_tiles[j],".geojson'",sep=""))
    }
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_19400.geojson'


[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[112] 1

[1] 20955

[1] "#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_20955.geojson'"


In [14]:
missing_tiles

integer(0)

In [54]:
library("sf")

Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE



In [60]:
install.packages("gpkg")
library("gpkg")

Warning message in install.packages("gpkg"):
“installation of package ‘gpkg’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in library("gpkg"): there is no package called ‘gpkg’


In [23]:
iso3_list <- world$ADM0_A3

for(i in 1:length(iso3_list)){
    
    iso3 <- iso3_list[i]
    country <- world[world$ADM0_A3 %in% iso3,]
    aoi <- country
    
    print(iso3)
    
#    if(!file.exists(paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))){
        
    if (!identical(crs(aoi), crs(G1deg_tiles))){aoi <- project(aoi, crs(G1deg_tiles))}
        intersecting <- intersect(G1deg_tiles, aoi)
          if (nrow(intersecting) == 0) {print('no intersecting G1deg_tiles found')
          } else {
            tileindex <- intersecting$id
          }
        #print(tileindex)
    
    tilenames <- rep("NA",length(tileindex))
    s3path <- rep("NA",length(tileindex))
    l4path <- rep("NA",length(tileindex))

    tileindex_df <- data.frame(tileindex, tilenames, s3path, l4path)
        for(i in 1:nrow(tileindex_df)){
            tileindex_i <- tileindex_df[i,1]
            tileindex_df[i,2] <- paste("/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
            tileindex_df[i,3] <- paste("https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
            tileindex_df[i,4] <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_",tileindex_i,"_L4A.gpkg",sep="")
        }
    write.csv(tileindex_df, paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))}
#}

[1] "IDN"
[1] "MYS"
[1] "CHL"
[1] "BOL"
[1] "PER"
[1] "ARG"
[1] "ESB"
[1] "CYP"
[1] "IND"
[1] "CHN"
[1] "ISR"
[1] "PSX"
[1] "LBN"
[1] "ETH"
[1] "SDS"
[1] "SOM"
[1] "KEN"
[1] "PAK"
[1] "MWI"
[1] "TZA"
[1] "SYR"
[1] "SOL"
[1] "FRA"
[1] "SUR"
[1] "GUY"
[1] "KOR"
[1] "PRK"
[1] "MAR"
[1] "SAH"
[1] "CRI"
[1] "NIC"
[1] "COG"
[1] "COD"
[1] "BTN"
[1] "UKR"
[1] "BLR"
[1] "NAM"
[1] "ZAF"
[1] "MAF"
[1] "SXM"
[1] "OMN"
[1] "UZB"
[1] "KAZ"
[1] "TJK"
[1] "LTU"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "BRA"
[1] "URY"
[1] "MNG"
[1] "RUS"
[1] "CZE"
[1] "DEU"
[1] "EST"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "LVA"


...

[1] "no intersecting G1deg_tiles found"
[1] "FRO"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "IOA"
[1] "IOT"
[1] "SGP"
[1] "NFK"
[1] "COK"
[1] "TON"
[1] "WLF"
[1] "WSM"
[1] "SLB"
[1] "TUV"
[1] "MDV"
[1] "NRU"
[1] "FSM"
[1] "SGS"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "FLK"
[1] "VUT"
[1] "NIU"
[1] "ASM"
[1] "PLW"
[1] "GUM"
[1] "MNP"
[1] "BHR"
[1] "CSI"
[1] "PGA"
[1] "CLP"
[1] "MAC"
[1] "ATC"
[1] "BJN"
[1] "SER"
[1] "SCR"
